In [1]:
import os
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from sklearn.cluster import KMeans
from collections import Counter
from sklearn.metrics import silhouette_samples

In [2]:
names = (
    'user_id',
    'check_in_time',
    'latitude',
    'longitude',
    'location_id',
    )
data = pd.read_csv("Gowalla/Gowalla_totalCheckins.txt", sep="\t", header=None, names = names, index_col=False, engine='python')

In [3]:
data.head()

,user_id,check_in_time,latitude,longitude,location_id
0,0,2010-10-19T23:55:27Z,30.235909,-97.795140,22847
1,0,2010-10-18T22:17:43Z,30.269103,-97.749395,420315
2,0,2010-10-17T23:42:03Z,30.255731,-97.763386,316637
3,0,2010-10-17T19:26:05Z,30.263418,-97.757597,16516
4,0,2010-10-16T18:50:42Z,30.274292,-97.740523,5535878


In [4]:
data.tail()

,user_id,check_in_time,latitude,longitude,location_id
6442887,196578,2010-06-11T13:32:26Z,51.742988,-0.488065,906885
6442888,196578,2010-06-11T13:26:45Z,51.746492,-0.490780,965121
6442889,196578,2010-06-11T13:26:34Z,51.741916,-0.496729,1174322
6442890,196585,2010-10-08T21:01:49Z,50.105516,8.571525,471724
6442891,196585,2010-10-07T17:39:18Z,50.027812,8.785098,4555073


In [5]:
# split column into multiple columns by delimiter 
data['date'] = pd.to_datetime(data['check_in_time']).dt.date
data['time'] = pd.to_datetime(data['check_in_time']).dt.time

In [6]:
data.drop(columns=['check_in_time'])

,user_id,latitude,longitude,location_id,date,time
0,0,30.235909,-97.795140,22847,2010-10-19,23:55:27
1,0,30.269103,-97.749395,420315,2010-10-18,22:17:43
2,0,30.255731,-97.763386,316637,2010-10-17,23:42:03
3,0,30.263418,-97.757597,16516,2010-10-17,19:26:05
4,0,30.274292,-97.740523,5535878,2010-10-16,18:50:42
...,...,...,...,...,...,...
6442887,196578,51.742988,-0.488065,906885,2010-06-11,13:32:26
6442888,196578,51.746492,-0.490780,965121,2010-06-11,13:26:45
6442889,196578,51.741916,-0.496729,1174322,2010-06-11,13:26:34
6442890,196585,50.105516,8.571525,471724,2010-10-08,21:01:49


In [11]:
df = pd.DataFrame(data)

In [12]:
df.isnull().values.any()

False

In [16]:
df = df.drop('check_in_time', 1)

In [17]:
df.head()

,user_id,latitude,longitude,location_id,date,time
0,0,30.235909,-97.795140,22847,2010-10-19,23:55:27
1,0,30.269103,-97.749395,420315,2010-10-18,22:17:43
2,0,30.255731,-97.763386,316637,2010-10-17,23:42:03
3,0,30.263418,-97.757597,16516,2010-10-17,19:26:05
4,0,30.274292,-97.740523,5535878,2010-10-16,18:50:42


In [ ]:
def get_spans(df, partition, scale=None):
    """
    :param        df: the dataframe for which to calculate the spans
    :param partition: the partition for which to calculate the spans
    :param     scale: if given, the spans of each column will be divided
                      by the value in `scale` for that column
    :        returns: The spans of all columns in the partition
    """
    spans = {}
    for column in df.columns:
        if column in categorical:
            span = len(df[column][partition].unique())
        else:
            span = df[column][partition].max()-df[column][partition].min()
        if scale is not None:
            span = span/scale[column]
        spans[column] = span
    return spans